**Корректность проверена на Python 3.6:**
+ gensim 3.4.0

# Пример использования библиотеки gensim для тематического моделирования
## Данная модель основана на байесовских методов машинного обучения

In [53]:
from gensim import corpora, models
import re

In [12]:
# Импортируем данные в формте UCI Bag of Words
data = corpora.UciCorpus("docword.xkcd.uci", "vocab.xkcd.uci")
dictionary = data.create_dictionary()

In [62]:
# обучение модель
num_topics = 2
%time ldamodel = models.ldamodel.LdaModel(data, id2word=dictionary, num_topics=num_topics, passes=20, alpha=1.25, eta=1.25)

Wall time: 140 ms


In [21]:
# Сохранение модели
ldamodel.save("ldamodel_xkcd")

In [22]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("ldamodel_xkcd")

In [77]:
# выводим топы слов
num_words = 3
for t, top_words in ldamodel.print_topics(num_topics=2, num_words=num_words):
    print("Topic", t, ":", top_words)

Topic 0 : 0.181*"b'industry'" + 0.114*"b'distance'" + 0.097*"b'co-financing'"
Topic 1 : 0.257*"b'subsidy'" + 0.103*"b'loan'" + 0.068*"b'support'"


In [25]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print(2**(-perplexity))

6.658405631341573


In [30]:
sum_words = 150
perp = ldamodel.bound(data)
2**(-perp/float(sum_words))

2.928036513941492

In [99]:
# Получение распределения тем для конкретного документа
# Индекс + 1 списка data соответствует id меры поддержки (нумерация мер с 1 до последней меры)
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.24752639), (1, 0.7524736)]

In [90]:
# Получение топовых слов по теме
# Индекс списка соответствует тематике
count_of_themes = 6
words_of_theme = []
for j in range(num_topics):
    df = [re.split('[^a-z]', sentence.lower().replace('\n', '')) for sentence in \
         ldamodel.print_topics(num_topics=num_topics, num_words=3)[j][1].split('+')]
    words_of_theme.append(list(set([word for sen in df for word in sen if len(word) > 2])))


### Получение запроса в текстовом формате

In [68]:
text_info = 'Subsidy finacial support'
data_list = text_info.split()
data_list_lower = []
for word in data_list:
    lw = word.lower().replace('\n', '')
    data_list_lower.append(lw)
words_user = list(set(data_list_lower))

In [97]:
list_of_themes = []
for i in range(len(words_user)):
    for j in range(len(words_of_theme)):
        for y in range(num_words):
            if words_user[i] == words_of_theme[j][y]:
                if ldamodel.get_document_topics(doc)[0][1] > ldamodel.get_document_topics(doc)[1][1]:
                     list_of_themes.append(ldamodel.get_document_topics(doc)[0])
                else:
                    list_of_themes.append(ldamodel.get_document_topics(doc)[1])


In [98]:
list_of_themes

[(1, 0.75247496), (1, 0.75250614)]